In [88]:
import pandas as pd
from IPython.display import Markdown
import prompts
import utils
import json

from importlib import reload
reload(utils)

<module 'utils' from '/usr/local/google/home/raulramos/projects/llmrouting/utils.py'>

In [89]:
models = ["gemini-2.5-flash-preview-05-20",
          'meta/llama-4-scout-17b-16e-instruct-maas']

modelA = models[0]
modelB = models[1]

modelA_response_col = f"{modelA}::response"
modelB_response_col = f"{modelB}::response"


In [90]:
data_file = 'dolly_15k_extended_small_3k.h5'
z = pd.read_hdf(data_file)
cols_response = [f"{model}::response" for model in models]
cols_code = [f"{model}::code" for model in models]

z = z[(z[cols_code] == 'ok').sum(axis=1) == 2]

z.shape

(3000, 9)

In [91]:
reload(prompts)
metric_prompts = {'groundness': prompts.groundness,
                  'text_quality': prompts.text_quality,
                 'summarization': prompts.summarization,
                 'verbosity': prompts.verbosity}

In [5]:
reload(utils)
reload(prompts)


<module 'prompts' from '/usr/local/google/home/raulramos/projects/llmrouting/prompts.py'>

In [6]:

zmetric_prompts = {k: pd.DataFrame(
        [{'prompt': v.format(prompt = zi.context + '\n\n' + zi.instruction , responseA = zi[modelA_response_col], responseB = zi[modelB_response_col]),
         'id': zi.name,
         'response': '',
         'code': 'pending'} for _,zi in z.iterrows()]) \
    for k,v in metric_prompts.items() }


In [7]:
data_file[:-3]

'dolly_15k_extended_small_3k'

In [8]:

te = utils.GenAI(project='genai-dev-454121', location='us-central1')
for metric_name, metric_prompts in zmetric_prompts.items():
    print ('metric', metric_name, flush=True)
    df = metric_prompts.copy()
    while (sum(df.code=='pending')>0):
        df_ok      = df[df.code=='ok'].copy()
        df_pending = df[df.code=='pending'].copy()
        df_rest    = df[~df.code.isin(['ok', 'pending'])].copy()
        print (f'  ok {len(df_ok)}, pending {len(df_pending)}, rest {len(df_rest)}', flush=True)  

        contents = te.generate_contents(df_pending['prompt'].values, model="gemini-2.5-flash-preview-05-20")

        df_pending['response'] = contents['response'].values
        df_pending['code'] = contents['code'].values
        df_pending['code'] = ['pending' if (i['response'] is None or 'RESOURCE_EXHAUSTED' in i['code'] or i['code'] == 'RefreshError') \
                                          else i["code"] for _,i in df_pending.iterrows()]

        df = pd.concat([df_ok, df_pending, df_rest])

        df.to_hdf(f'{data_file[:-3]}_{metric_name}.h5', key='main')

        


metric groundness
  ok 0, pending 3000, rest 0


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed: 10.4min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 15.9min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed: 23.0min
[Parallel(n_jobs=-1)]: Done 866 tasks      | elapsed: 31.1min
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed: 40.9min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed: 51.1min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed: 64.2min
[Parallel(n_jobs=-1)]: Done 2162 tasks      | elapsed: 78.1min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed: 92.8min
[Parallel(n_jobs=-1)]: Done 3000 out of 3000 | elapsed: 107.7min finished
/tmp/ipykernel_2787853/250754072.py:20: PerformanceWarning: 
your performance may suffer as PyTables 

  ok 2997, pending 2, rest 1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


  ok 2998, pending 1, rest 1


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


  ok 2998, pending 1, rest 1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


metric verbosity
  ok 0, pending 3000, rest 0


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed: 14.5min
[Parallel(n_jobs=-1)]: Done 866 tasks      | elapsed: 20.0min
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed: 26.2min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed: 33.2min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed: 41.1min
[Parallel(n_jobs=-1)]: Done 2162 tasks      | elapsed: 49.8min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed: 59.1min
[Parallel(n_jobs=-1)]: Done 3000 out of 3000 | elapsed: 68.7min finished


In [10]:
df

,prompt,id,response,code
0,\n# Instruction\nYou are an expert evaluator. ...,12052,## Evaluation\n\n### STEP 1: Analyze Response ...,ok
1,\n# Instruction\nYou are an expert evaluator. ...,10047,## Evaluation\n\n### STEP 1: Analyze Response ...,ok
2,\n# Instruction\nYou are an expert evaluator. ...,14743,## Evaluation\n\n### STEP 1: Analyze Response ...,ok
3,\n# Instruction\nYou are an expert evaluator. ...,8131,"```json\n{\n ""explanation"": ""STEP 1: Analyze R...",ok
4,\n# Instruction\nYou are an expert evaluator. ...,11010,## Evaluation\n\n### STEP 1: Analyze Response ...,ok
5,\n# Instruction\nYou are an expert evaluator. ...,2258,## Evaluation\n\n**STEP 1: Analyze Response A ...,ok
6,\n# Instruction\nYou are an expert evaluator. ...,421,"**Explanation:**\n\nThe user prompt is ""Write ...",ok
7,\n# Instruction\nYou are an expert evaluator. ...,2215,Explanation:\nSTEP 1: Analyze Response A based...,ok
8,\n# Instruction\nYou are an expert evaluator. ...,10586,* **STEP 1: Analyze Response A based on the ...,ok
9,\n# Instruction\nYou are an expert evaluator. ...,4230,* **STEP 1: Analyze Response A based on the ...,ok


In [86]:
reload(utils)
r = {}
for m in metric_prompts.keys():
    z = pd.read_hdf(f'{data_file[:-3]}_{m}.h5')
    r[m] = pd.Series([utils.extract_json_rating(i) for i in z.response.values]).value_counts()
r = pd.DataFrame(r).fillna(0)

In [87]:
r

,groundness,text_quality,summarization,verbosity
-1.00,804.0,2256.0,1764.0,1967.0
-0.80,1.0,0.0,0.0,0.0
-0.75,5.0,0.0,0.0,0.0
-0.50,21.0,7.0,0.0,0.0
-0.25,0.0,1.0,0.0,0.0
-0.20,3.0,6.0,0.0,0.0
-0.10,0.0,12.0,0.0,0.0
0.00,1568.0,282.0,162.0,139.0
0.20,1.0,1.0,0.0,0.0
0.50,6.0,42.0,0.0,0.0


In [98]:
for m in metric_prompts.keys():
    z = pd.read_hdf(f'{data_file[:-3]}_{m}.h5')
    break
    

In [97]:
f'{data_file[:-3]}_{m}.h5'

'dolly_15k_extended_small_3k_groundness.h5'

In [99]:
z

,prompt,id,response,code
0,\n# Instruction\nYou are an expert evaluator. ...,4132,## Evaluation\n\n### STEP 1: Analyze Response ...,ok
1,\n# Instruction\nYou are an expert evaluator. ...,1325,Explanation:\nSTEP 1: Analyze Response A based...,ok
2,\n# Instruction\nYou are an expert evaluator. ...,274,explanation:\nResponse A is fully grounded as ...,ok
3,\n# Instruction\nYou are an expert evaluator. ...,1305,## Evaluation\n\n### STEP 1: Analyze Response ...,ok
4,\n# Instruction\nYou are an expert evaluator. ...,4697,## Evaluation\n\n### STEP 1: Analyze Response ...,ok
...,...,...,...,...
2998,\n# Instruction\nYou are an expert evaluator. ...,8697,## Evaluation\n\n### STEP 1: Analyze Response ...,ok
2999,\n# Instruction\nYou are an expert evaluator. ...,7625,"```json\n{\n ""rating"": 0\n}\n```\n\n### Explan...",ok
1550,\n# Instruction\nYou are an expert evaluator. ...,14970,## Evaluation\n\n### STEP 1: Analyze Response ...,ok
2146,\n# Instruction\nYou are an expert evaluator. ...,493,## Evaluation\n\n### STEP 1: Analyze Response ...,ok
